In [26]:
%autoreload 2

import sys
import os
import ccxt

sys.path.append(os.path.dirname(os.path.abspath('')))

from tensortrade.environments import TradingEnvironment
from tensortrade.exchanges.live import CCXTExchange

binance = ccxt.binance({ 'apiKey': API_KEY, 'secret': SECRET_KEY })

exchange = CCXTExchange(exchange=binance)

exchange.next_observation()

[autoreload of tensortrade.exchanges.live.ccxt_exchange failed: Traceback (most recent call last):
  File "/Users/adam/anaconda3/envs/tensortrade/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/adam/anaconda3/envs/tensortrade/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/adam/anaconda3/envs/tensortrade/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 387, in update_generic
    update(a, b)
  File "/Users/adam/anaconda3/envs/tensortrade/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 357, in update_class
    update_instances(old, new)
  File "/Users/adam/anaconda3/envs/tensortrade/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 312, in update_instances
    update_instances(old, new, obj.__dict__, visited)
  File "/Users/adam/anaconda3/envs/tens

[[0, 0.01886, 1.013, 0.019105179999999996],
 [0, 0.018861, 0.193, 0.003640173],
 [1, 0.018859, 1.648, 0.031079632],
 [0, 0.01886, 0.297, 0.005601419999999999],
 [0, 0.018861, 0.106, 0.001999266],
 [0, 0.018861, 0.216, 0.004073976],
 [0, 0.018862, 1.26, 0.02376612],
 [0, 0.018862, 0.04, 0.0007544800000000001],
 [1, 0.01886, 0.104, 0.0019614399999999996],
 [0, 0.018863, 0.009, 0.000169767],
 [0, 0.018861, 1.368, 0.025801848000000002],
 [1, 0.018859, 0.264, 0.004978776000000001],
 [1, 0.018858, 0.106, 0.001998948],
 [0, 0.018859, 0.694, 0.013088146],
 [1, 0.018857, 9.749, 0.183836893],
 [1, 0.018857, 0.056, 0.001055992],
 [1, 0.018857, 0.056, 0.001055992],
 [1, 0.018857, 0.307, 0.005789099],
 [0, 0.01886, 0.099, 0.00186714],
 [1, 0.018857, 2.474, 0.046652218],
 [1, 0.018856, 0.106, 0.001998736],
 [1, 0.018853, 1.0, 0.018853],
 [0, 0.018856, 0.099, 0.0018667440000000003],
 [0, 0.018855, 2.388, 0.04502574],
 [0, 0.018856, 0.482, 0.009088592],
 [0, 0.018859, 0.099, 0.001867041],
 [1, 0.01885